In [1]:
from openai import OpenAI
import os
import json
from pydantic import BaseModel,Field
from langchain.llms import OpenAI
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage
from tqdm import tqdm
import re
import textwrap
from langgraph.graph import StateGraph, END
from typing import TypedDict, Optional
from openai import OpenAI
from pydantic import BaseModel
import traceback
from typing import List, Union
from langgraph.graph import StateGraph, END
from typing import TypedDict, Optional
from concurrent.futures import ThreadPoolExecutor, as_completed
from langsmith import traceable, trace
from langsmith import Client, traceable, evaluate
from langsmith import evaluate, Client
import langsmith as ls
from preprocess_data import prepare_qa_input_with_answer_filter,standardize_item


In [2]:
file_path = '../data/GSM8K/test.jsonl'

with open(file_path, "r", encoding="utf-8") as f:
    gsm8k = [json.loads(line) for line in f]

In [3]:
folder_path = "../dataset_langsmith/"
filename="tatqa_filtered.json"
file_path = os.path.join(folder_path, filename)
with open(file_path, "r", encoding="utf-8") as f:
    raw_data_tatqa = json.load(f)
tatqa = prepare_qa_input_with_answer_filter(raw_data_tatqa)

In [4]:
folder_path = "../dataset_langsmith/"
filename="tabmwp_filtered.json"
file_path = os.path.join(folder_path, filename)
with open(file_path, "r", encoding="utf-8") as f:
    tabmwp = json.load(f)

In [5]:
name= "gsm8k" # tatqa, tabmwp
length_test= 300 # số lượng mẫu muốn test
if name == "gsm8k":
   DATA=gsm8k 
   name_model="GSM8K"
elif name == "tatqa":
   DATA=tatqa
   name_model="TATQA"
else:
   DATA=tabmwp
   name_model="TABMWP"

In [6]:
load_dotenv()
model = init_chat_model("gpt-4.1-mini", model_provider="openai", temperature=0.2)

In [7]:
class State(TypedDict):
    question: str
    context: Optional[str]
    program: Optional[str]
    result: Optional[str]
    final_answer: Optional[str]
    error: Optional[str]
class IntermediateProgram(BaseModel):
    program: str


In [ ]:
few_shot_gsm8k='''
Question: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?
```
def solver():
    """Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?"""
    toys_initial = 5
    mom_toys = 2
    dad_toys = 2
    total_received = mom_toys + dad_toys
    total_toys = toys_initial + total_received
    return total_toys
```

Question: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
```
def solver():
    """Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?"""
    leah_chocolates = 32
    sister_chocolates = 42
    total_chocolates = leah_chocolates + sister_chocolates
    chocolates_eaten = 35
    chocolates_left = total_chocolates - chocolates_eaten
    return chocolates_left
```
    

Question:  Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?
```
def solver():
    """ Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?"""
    golf_balls_initial = 58
    golf_balls_lost_tuesday = 23
    golf_balls_lost_wednesday = 2
    golf_balls_left = golf_balls_initial - golf_balls_lost_tuesday - golf_balls_lost_wednesday
    return golf_balls_left
```
'''.strip()
few_shot_tatqa='''
Context: |  |  | Three Months Ended |  | % Variation |  |\n| --- | --- | --- | --- | --- | --- |\n|  | December 31, 2019 | September 29, 2019 | December 31, 2018 | Sequential | Year-Over-Year |\n|  |  |  | (Unaudited, in millions) |  |  |\n| Selling, general and administrative expenses | $(285) | $(267) | $(285) | (6.3)% | 0.4% |\n| Research and development expenses | (387) | (362) | (345) | (7.0) | (12.3) |\n| Total operating expenses | $(672) | $(629) | $(630) | (6.7)% | (6.6)% |\n| As percentage of net revenues | (24.4)% | (24.7)% | (23.8)% | +30 bps | -60 bps |\n
Question: What is the increase/ (decrease) in total operating expenses from the period December 31, 2018 to 2019?
```
def solver():
    """What is the increase/ (decrease) in total operating expenses from the period December 31, 2018 to 2019?"""
    expenses_2019 = 672
    expenses_2018 = 630
    # Tính mức tăng
    answer = expenses_2019 - expenses_2018
    return answer 
```

Context: Property, plant and equipment at April 30, 2019 and 2018, consisted of the following (in thousands):\n\n|  | 2019 | 2018 |\n| --- | --- | --- |\n| Buildings and building improvements | $2,692 | $2,790 |\n| Machinery, equipment and furniture | 57,157 | 57,503 |\n|  | 59,849 | 60,293 |\n| Less, accumulated depreciation | (46,811 ) | (46,166) |\n|  | $13,038 | $ 14,127 |\n
Question: What is the average value of buildings and building improvements in 2018 and 2019?
```
def solver():
    """What is the average value of buildings and building improvements in 2018 and 2019?"""
    value_2019 = 2692
    value_2018 = 2790
    answer = (value_2019 + value_2018) / 2
    return answer
```

Context:
The funded status of our postretirement health care and other defined benefit plans, which is recognized in other long-term liabilities in our consolidated balance sheets, was as follows (in millions):
    |                          | April 26, 2019 | April 27, 2018 |
    |--------------------------|----------------|----------------|
    | Fair value of plan assets| $31 million    | $25 million    |
    | Benefit obligations      | $(61) million  | $(53) million  |
    | Unfunded obligations     | $(30) million  | $(28) million  |
Question: What was the change in the fair value of plan assets between 2018 and 2019?
```
def solver():
    """What was the change in the fair value of plan assets between 2018 and 2019?"""
    fair_value_2019 = 31
    fair_value_2018 = 25
    change = fair_value_2019 - fair_value_2018
    return change
```
'''.strip()
few_shot_tabmwp='''
Context:
| Number of times | Frequency |
|-----------------|-----------|
| 0               | 1         |
| 1               | 18        |
| 2               | 12        |
| 3               | 13        |
| 4               | 0         |
Question: How many customers are there in all?
```
def solver():
    """How many customers are there in all?"""
    frequencies = [1, 18, 12, 13, 0]
    ans = sum(frequencies)
    return ans
```
'''.strip()

if name=="gsm8k":
    select_fewshot=few_shot_gsm8k
elif name=="tatqa":
    select_fewshot=few_shot_tatqa
else:
    select_fewshot=few_shot_tabmwp

In [9]:
def pot_node(state: State) -> State:
    context_str = f"# Context:\n{state['context']}\n" if state.get("context") else ""
    pot_messages = [
        SystemMessage('''You will write python program to solve math problems. You will only write code blocks.'''),
        HumanMessage(content=f'''
{select_fewshot}
How about this question?
{context_str}
# Question: {state["question"]}
# Answer this question by implementing a solver() function.
# Include a final answer as a single number, no units or symbols.
# 'CALL' the solver() function and then 'MUST' assign the variale 'result'.
# Before returning the final result, DOUBLE-CHECK each variable assignment and calculation to ensure they match the problem statement.
'''.strip())]

    model_pot = model.with_structured_output(IntermediateProgram)
    model_invoke=model_pot.invoke(pot_messages)
    code=model_invoke.program
    return {**state, "program": code}

def exec_node(state: State) -> State:
    try:
        exec_globals = {}
        exec(state["program"], {}, exec_globals)
        result = exec_globals.get("result", None)
        if result is None:
            raise ValueError("Missing `result`")
        return {**state, "result": str(result), "error": None}
    except Exception as e:
        return {**state, "result": None, "error": str(e)}

def check_eos(state: State) -> bool:
    if state["error"] is None:
        return True
    else:
        return False
def write_final_answer_node(state:State)->State:

    if state["error"] is None:
        result=str(state["result"])
    else:
        result=str(9999)
    return {**state,"final_answer":result}


builder = StateGraph(State)
builder.add_node("PoT", pot_node)
builder.add_node("Exec", exec_node)

builder.add_node("write_final_answer",write_final_answer_node)

builder.set_entry_point("PoT")
builder.add_edge("PoT", "Exec")
builder.add_edge("Exec", "write_final_answer")
builder.add_edge("write_final_answer",END)
graph = builder.compile()


In [10]:
def extract_ground_truth(answer, dataset_type):
    if dataset_type == "gsm8k":
        match = re.search(r"####\s*([\d,./]+)", answer)
        if match:
            return match.group(1).replace(",", "").strip()
        return answer.strip()
    elif dataset_type == "tatqa":
        # Có thể là list hoặc chuỗi, lấy phần đầu tiên nếu là list
        if isinstance(answer, list):
            ans = str(answer[0]).strip()
        else:
            ans = str(answer).strip()
        # Nếu là dạng [2019] hoặc ["2019"], loại bỏ ngoặc vuông và dấu nháy
        ans = re.sub(r'^[\[\"]*([\d\-\.\/]+)[\]\"]*$', r'\1', ans)
        # Nếu là phân số thì giữ lại dấu "/"
        if '/' in ans:
            ans = re.sub(r"[^-\d/\.]", "", ans)
        else:
            ans = re.sub(r"[^-\d\.]", "", ans)
        return ans
    else:
        return str(answer).strip()
    
def compare_answers(predicted: str, actual: str, eps: float = 1e-3) -> bool:
    try:
        pred = round(float(predicted.strip()))
        act = round(float(actual.strip()))
        return abs(pred - act) < eps
    except ValueError:
        return predicted.strip().lower() == actual.strip().lower()



In [11]:

def process_item(item, dataset_type):
    question = item["question"]
    context = item.get("context", "")
    true_answer = extract_ground_truth(item["answer"], dataset_type)
    try:
        # Nếu pipeline/graph của bạn cần context thì truyền vào, nếu không thì chỉ truyền question
        if context:
            result = graph.invoke({"question": question, "context": context})
        else:
            result = graph.invoke({"question": question})
        return {
            "question": question,
            "program": result.get("program", ""),
            "true_answer": true_answer,
            "predicted_answer": result["final_answer"],
            "context": context,
            "correct": compare_answers(result["final_answer"], true_answer)
        }
    except Exception as e:
        return {"error": str(e), "question": question}


dataset = []
for item in DATA:
    dataset.extend(standardize_item(item, f"{name}"))

results = []
correct = 0
total = len(dataset[:length_test])
with ThreadPoolExecutor(max_workers=5) as executor:
    futures = [executor.submit(process_item, item, f"{name}") for item in dataset[:length_test]]
    for future in tqdm(as_completed(futures), total=total):
        result = future.result()
        if "error" not in result:
            results.append(result)
            if result["correct"]:
                correct += 1
        else:
            print(f"Error on question: {result['question'][:60]}... => {result['error']}")
accuracy = correct / total * 100
print(f"Accuracy: {accuracy:.2f}% ({correct}/{total})")


  0%|          | 0/300 [00:00<?, ?it/s]Failed to send compressed multipart ingest: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Too many requests: tenant exceeded usage limits: Monthly unique traces usage limit exceeded"}\n')trace=04af226d-2938-4368-85f6-59eeab4b3a0d,id=04af226d-2938-4368-85f6-59eeab4b3a0d; trace=04af226d-2938-4368-85f6-59eeab4b3a0d,id=3e0887bf-6a76-4564-b0b5-876f4ffa73ea; trace=04af226d-2938-4368-85f6-59eeab4b3a0d,id=1619899a-3c25-40b1-a416-837d9fc911a2; trace=04af226d-2938-4368-85f6-59eeab4b3a0d,id=a120ad8e-2c05-432f-b41e-9442df1f32f2
Failed to send compressed multipart ingest: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multip

Accuracy: 97.67% (293/300)


In [12]:
output_path = "PaL_results.json"
os.makedirs(os.path.dirname(output_path) or ".", exist_ok=True)
def custom_encoder(obj):
    if hasattr(obj, "model_dump"):
        return obj.model_dump()
    elif hasattr(obj, "dict"):
        return obj.dict()
    else:
        raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")

wrong_answers = [r for r in results if not r.get("correct", False)]

# Ghi file JSON
try:
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(wrong_answers, f, ensure_ascii=False, indent=2, default=custom_encoder)
    print(f"Đã lưu {len(wrong_answers)} kết quả sai vào {output_path}")
except TypeError as e:
    print(f"Lỗi khi ghi file JSON: {e}")


Đã lưu 7 kết quả sai vào PaL_results.json


In [ ]:
import math
def run_graph(inputs: dict):
    # Chuẩn bị state đầu vào cho graph
    state = {
        "question": inputs["question"],
        "context": inputs.get("context", ""),
        "error": None,
    }
    # Chạy graph
    final_state = graph.invoke(state)
    # Lấy kết quả cuối cùng
    result = final_state
    # Nếu bạn muốn log token, có thể lấy từ state (nếu đã lưu ở node PoT)
    token_info = {
        "input_tokens": final_state.get("input_tokens", None),
        "output_tokens": final_state.get("output_tokens", None),
        "total_tokens": final_state.get("total_tokens", None),
    }
    return token_info, result
@traceable(run_type="tool")
def compare_result(inputs: dict, reference_outputs: dict, outputs: dict):
    # Lấy đúng trường kết quả
    reference_response = extract_ground_truth(reference_outputs["answer"],f"{name}")
    run_response = outputs.get("final_answer")
    # Đảm bảo là string
    reference_response = str(reference_response).strip()
    run_response = str(run_response).strip()
    # So sánh số
    try:
        score = math.isclose(float(reference_response), float(run_response), rel_tol=1e-3)
    except Exception:
        score = (reference_response == run_response)
    return {"key": "is_correct", "score": int(score)}

@traceable(run_type="chain")
def target_function(inputs: dict):
    token, result = run_graph(inputs)
    # Nếu dùng LangSmith, log token vào run tree
    rt = ls.get_current_run_tree()
    if token["input_tokens"] is not None:
        rt.metadata["input_tokens"] = token["input_tokens"]
        rt.metadata["output_tokens"] = token["output_tokens"]
        rt.metadata["total_tokens"] = token["total_tokens"]
    return result


client = Client()
evaluate(
    target_function,
    data=client.list_examples(dataset_name=f"{name_model}", splits=["base"]),
    evaluators=[compare_result],
    experiment_prefix=f"{name_model} - Test Dataset"
)


Failed to send compressed multipart ingest: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Too many requests: tenant exceeded usage limits: Monthly unique traces usage limit exceeded"}\n')trace=bf9903ef-3cb6-46d4-b602-5517777836cb,id=bd1893b7-9f13-4857-8c05-0b01c0368054; trace=bf9903ef-3cb6-46d4-b602-5517777836cb,id=e89ead84-7993-492b-b854-5fbcad1d4b01; trace=bf9903ef-3cb6-46d4-b602-5517777836cb,id=e89ead84-7993-492b-b854-5fbcad1d4b01; trace=bf9903ef-3cb6-46d4-b602-5517777836cb,id=7d5626ac-4937-4b58-86c6-4499bc1571e5; trace=bf9903ef-3cb6-46d4-b602-5517777836cb,id=9cf0d5cd-6e99-4344-b4d5-4aec4c258aaf; trace=bf9903ef-3cb6-46d4-b602-5517777836cb,id=361ddbbe-81d6-4c98-8c9b-c534d873f293; trace=bf9903ef-3cb6-46d4-b602-5517777836cb,id=361ddbbe-81d6-4c98-8c9b-c534d873f293; trace=bf9903ef-3cb6-46d4-b602-5517777836cb

View the evaluation results for experiment: 'GSM8K - Test Dataset-d2d40235' at:
https://smith.langchain.com/o/e7b5e917-6c40-46ad-a54b-83be84870fd4/datasets/a0f9251e-0ade-4046-a15d-3eb36e10746f/compare?selectedSessions=6f274ca1-785e-43ae-a386-2fcd35eeb070




0it [00:00, ?it/s]

Failed to send compressed multipart ingest: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Too many requests: tenant exceeded usage limits: Monthly unique traces usage limit exceeded"}\n')trace=b90e36ba-1e88-41a0-922a-f29d0dc02955,id=b90e36ba-1e88-41a0-922a-f29d0dc02955; trace=b90e36ba-1e88-41a0-922a-f29d0dc02955,id=bfa6b8e2-6211-4d41-ba16-3022d885ff0e; trace=b90e36ba-1e88-41a0-922a-f29d0dc02955,id=79683017-5408-41fa-8004-7374cf49cb9e; trace=b90e36ba-1e88-41a0-922a-f29d0dc02955,id=9f93ce75-5c33-4f79-9ab3-656571d2515f; trace=b90e36ba-1e88-41a0-922a-f29d0dc02955,id=498dc0f6-a626-4dd0-8d92-668aa0052649
Failed to send compressed multipart ingest: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url